In [57]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import models.TextRCNN as model
from utils import DataConfig

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [58]:
data_config = DataConfig('100d', 'word2vec')
model_config = model.ModelConfig()

In [97]:
class Model(nn.Module):
    def __init__(self, model_config, data_config):
        super(Model, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            data_config.embedding_pretrained,
            freeze=False) if data_config.embedding_pretrained is not None else nn.Embedding(data_config.n_vocab,
                                                                                            data_config.embed,
                                                                                            padding_idx=data_config.n_vocab - 1)
        self.lstm = nn.LSTM(data_config.embed, model_config.hidden_size, model_config.num_layers,
                            bidirectional=True, batch_first=True, dropout=model_config.dropout)
        self.fc = nn.Linear(model_config.hidden_size * 2 + data_config.embed, data_config.num_classes)

    def forward(self, x):
        embed = self.embedding(x)  # [batch_size, seq_len, embeding]=[64, 32, 64]
        out, _ = self.lstm(embed)
        out = torch.cat((embed, out), 2)
        out = F.relu(out)
        out = out.permute(0, 2, 1)
        _, top_indices = torch.topk(out, k=1, dim=2)
        out = torch.gather(out, 1, top_indices).squeeze()
        out = self.fc(out)
        return out



In [98]:
model = Model(model_config, data_config).to(data_config.device)

In [99]:
model(torch.randint(1, 10, [2, 30]).to(data_config.device))

tensor([[ 0.2476,  0.1735, -0.1111,  0.0347, -0.0054],
        [ 0.0092,  0.0929, -0.1043,  0.0915,  0.0131]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [90]:
def init_network(model, method='xavier', exclude='embedding'):
    for name, w in model.named_parameters():
        if exclude not in name:  # 如果不是嵌入层
            if 'weight' in name:  # weight 三种初始化方式
                if method == 'xavier' and len(w.size() < 2):
                    nn.init.xavier_normal_(w)
                elif method == 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:  # bias 置0
                nn.init.constant_(w, 0)
            else:
                pass

In [23]:
init_network(model)

In [78]:
from torchinfo import summary

summary(model, input_size=(1, 30), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [5]                       --
├─Embedding: 1-1                         [1, 30, 100]              72,700
├─LSTM: 1-2                              [1, 30, 512]              2,310,144
├─MaxPool1d: 1-3                         [1, 612, 1]               --
├─Linear: 1-4                            [5]                       3,065
Total params: 2,385,909
Trainable params: 2,385,909
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 69.39
Input size (MB): 0.00
Forward/backward pass size (MB): 0.15
Params size (MB): 9.54
Estimated Total Size (MB): 9.69

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
df = pd.read_csv('./ship_data/experiment_data.csv')  # 替换为你的CSV文件路径

# 划分数据集
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=3407)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=3407)

# 保存划分后的数据集为新的CSV文件
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)


In [100]:
import torch
import torch.nn.functional as F

# 假设你的输入是一个大小为[2, 30, 100]的tensor
input_tensor = torch.randn(2, 30, 100)

# 执行最大池化操作，保留最大的两个值
output_tensor = F.max_pool1d(input_tensor, kernel_size=2, stride=1)

print(output_tensor.shape)  # 输出应为[2, 2, 100]


torch.Size([2, 30, 99])
